In [1]:
##### 5.3 기술 통계 계산과 요약 #####

In [3]:
# pandas는 일반적인 수학 메서드와 통계 메서드를 가지고 있다.
# 이 메서드의 대부분은 단일 값을 구하는 '축소' 혹은 '요약 통계' 범주에 속한다.
# pandas 메서드는 numpy와 달리 처음부터 누락된 데이터를 제외하도록 설계되었다.

import pandas as pd
import numpy as np

In [5]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]], 
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])

In [6]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [7]:
df.sum() # 컬럼의 합

one    9.25
two   -5.80
dtype: float64

In [9]:
df.sum(axis=1)  # 로우의 합

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [10]:
# 전체 로우나 컬럼의 값이 NA가 아니라면 NA값은 제외되고 계산된다.
# skipna 옵션으로 조정할 수 있다.

df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

In [12]:
# idxmin이나 idxmax : 최솟값이나 최댓값이 가지고 있는 색인값과 같은 간접 통계를 반환한다.

df.idxmax()

one    b
two    d
dtype: object

In [14]:
# 누산 (누적합)

df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


In [15]:
# 축소나 누산이 아닌 다른 종류의 메서드로 describe가 있다.
# 한 번에 여러 개의 통계 결과를 만들어낸다.

df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


In [16]:
# 수치데이터가 아닐 경우 describe는 다른 요약 통계를 생성한다.

obj = pd.Series(['a', 'a', 'b', 'c'] * 4)

In [17]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

In [18]:
##### 5.3.1 상관관계와 공분산 #####

In [1]:
# 상관관계나 공분산 같은 요약 통계 계산은 두 쌍의 인자를 필요로 한다.
# pandas_datareader 모듈을 이용해 주가 정보를 다운로드하자

import pandas_datareader.data as web

In [4]:
all_data = {ticker: web.get_data_yahoo(ticker)
           for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}
price = pd.DataFrame({ticker: data['Adj Close']
                     for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume']
                      for ticker, data in all_data.items()})

In [5]:
returns = price.pct_change()

In [6]:
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2021-02-02,0.006337,-0.009126,-0.000584,0.013759
2021-02-03,-0.007778,-0.002679,0.014571,0.073961
2021-02-04,0.025758,0.015950,-0.004074,-0.003720
2021-02-05,-0.003098,0.006363,0.000785,0.017276
2021-02-08,0.001097,0.014944,0.001115,-0.002426


In [7]:
returns['MSFT'].corr(returns['IBM'])

0.5418188168400222

In [8]:
returns['MSFT'].cov(returns['IBM'])

0.00015461195181202503

In [10]:
# MSFT는 파이썬 속성 이름 규칙에 어긋나지 않으므로 좀 더 편리한 문법으로 컬럼 선택이 가능하다.

returns.MSFT.corr(returns.IBM)

0.5418188168400222

In [11]:
# DataFrame에서 corr와 cov 메서드는 DataFrame 행렬에서 상관관계와 공분산을 계산한다.

returns.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.454542,0.709510,0.650833
IBM,0.454542,1.000000,0.541819,0.506792
MSFT,0.709510,0.541819,1.000000,0.778115
GOOG,0.650833,0.506792,0.778115,1.000000


In [12]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.000358,0.000141,0.000233,0.000206
IBM,0.000141,0.000270,0.000155,0.000139
MSFT,0.000233,0.000155,0.000301,0.000225
GOOG,0.000206,0.000139,0.000225,0.000279


In [13]:
# DataFrame의 corrwith 메서드를 사용하면 다른 Series나 DataFrame과의 상관관계를 계산한다.
# Series를 넘기면 각 컬럼에 대해 계산한 상관관계를 담고 있는 Series를 반환한다.

returns.corrwith(returns.IBM)

AAPL    0.454542
IBM     1.000000
MSFT    0.541819
GOOG    0.506792
dtype: float64

In [15]:
# DataFrame을 넘기면 맞아떨어지는 컬럼 이름에 대한 상관관계를 계산한다.

returns.corrwith(volume)

AAPL   -0.071542
IBM    -0.110410
MSFT   -0.085393
GOOG   -0.134714
dtype: float64

In [16]:
# axis='columns' 각 컬럼에 대한 상관관계과 공분산 계산
# 데이터는 상관관계 계산 전 색인의 이름순으로 정렬됨

In [17]:
##### 5.3.2 유일값, 값 세기, 멤버십 #####

In [18]:
# 1차원 Series에 담긴 값의 정보 추출

obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

In [19]:
# unique 메서드는 중복되는 값을 제거하고 유일값만 담고 있는 Series를 반환한다.

uniques = obj.unique()

In [20]:
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

In [21]:
# 유일값은 정렬된 순서대로 반환되지 않지만 필요하다면 uniques.sort()를 이용해 정렬할 수 있다.
# value_counts 메서드는 Series에서 도수를 계산하여 반환한다.

obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

In [22]:
# value_counts에서 반환하는 Series는 담고 있는 값을 내림차순으로 정렬한다.
# value_counts 메서드는 pandas의 최상위 메서드로 어떤 배열이나 순차 자료구조에서도 사용할 수 있다.

In [23]:
pd.value_counts(obj.values, sort=False)

c    3
a    3
b    2
d    1
dtype: int64

In [24]:
# isin 메서드는 어떤 값이 Series에 존재하는지 나타내는 불리언 벡터를 반환한다.
# Series나 DataFrame의 컬럼에서 값을 골라내고 싶을 때 유용하게 사용할 수 있다.

obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [25]:
mask = obj.isin(['b', 'c'])

In [26]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

In [27]:
# isin과 관련있는 Index.get_indexer 메서드는 여러 값이 들어 있는 배열에서 유일한 값의 색인 배열을 구할 수 있다.
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])

In [28]:
unique_vals = pd.Series(['c', 'b', 'a'])

In [30]:
pd.Index(unique_vals).get_indexer(to_match)

array([0, 2, 1, 1, 0, 2], dtype=int32)

In [31]:
# DataFrame의 여러 컬럼에 대해 히스토그램을 구해야 하는 경우가 있다.

data = pd.DataFrame({'Qu1': [1, 3, 4, 3, 4],
                    'Ou2': [2, 3, 1, 2, 3],
                    'Ou3': [1, 5, 2, 4, 4]})

In [32]:
data

,Qu1,Ou2,Ou3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [33]:
result = data.apply(pd.value_counts).fillna(0)

In [34]:
result

,Qu1,Ou2,Ou3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0
